# Spherepop: Full Calculus Tutorial Notebook

This notebook builds a pedagogical but rigorous “full calculus” reference in three layers.

First, it defines an abstract **merge–collapse** calculus for regions, where values are geometric regions represented discretely as finite sets of atoms. Merge is union. Collapse is canonicalization under an equivalence relation that identifies atoms. Second, it presents an **operational semantics** as a deterministic normalization function together with a nondeterministic small-step reducer, and empirically checks confluence “up to collapse” by random reduction orders. Third, it shows how this calculus interfaces cleanly with the **Spherepop OS kernel** event log model (POP, MERGE, LINK, UNLINK, COLLAPSE, LABEL) by treating the kernel as an authoritative replay engine and the calculus as a compositional view language over replayed state.


## 0. Setup

We will use only standard Python plus matplotlib for optional visuals. The “geometry” here is discrete and finite so it is easily visualized and audited in a Jupyter tutorial. The intention is not to replace continuous geometry, but to provide a reference semantics that is deterministic, replayable, and demonstrates the invariants the monographs emphasize: idempotence, associativity up to canonicalization, and the separation between authoritative events and derived views.


In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Dict, Iterable, List, Optional, Set, Tuple, Union
from collections import defaultdict
import random
import math


## 1. Equivalence and Canonical Representatives (Collapse Data)

Collapse is modeled as quotienting by an equivalence relation. Concretely, an atom is mapped to a canonical representative. A collapsed region is the set of representatives of its atoms.

This choice makes collapse a true information-losing operator: distinct atoms can become indistinguishable after collapse, while downstream computation remains well-defined.


In [ ]:
class UnionFind:
    def __init__(self):
        self.parent: Dict[str, str] = {}

    def add(self, x: str) -> None:
        if x not in self.parent:
            self.parent[x] = x

    def find(self, x: str) -> str:
        if x not in self.parent:
            self.add(x)
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, a: str, b: str) -> None:
        ra, rb = self.find(a), self.find(b)
        if ra != rb:
            self.parent[rb] = ra

    def copy(self) -> "UnionFind":
        u = UnionFind()
        u.parent = dict(self.parent)
        return u


## 2. Regions, MERGE, and COLLAPSE

A region is represented as a finite set of atoms. MERGE is union. COLLAPSE is canonicalization under a chosen quotient structure.

The central semantic composite used throughout is:

\[
\mathrm{MERGE}(A,B) := \mathrm{COLLAPSE}(A \cup B).
\]

This implements the idea “merge as union followed by collapse.”


In [ ]:
Region = frozenset[str]

def region(*atoms: str) -> Region:
    return frozenset(atoms)

def collapse_region(R: Region, q: UnionFind) -> Region:
    return frozenset(q.find(a) for a in R)

def merge_region(A: Region, B: Region, q: UnionFind) -> Region:
    return collapse_region(A.union(B), q)


## 3. Algebraic Laws (Executable)

In this discrete reference semantics, several laws become straightforward to test.

Idempotence: MERGE(A, A) = COLLAPSE(A).  
Commutativity: MERGE(A, B) = MERGE(B, A).  
Associativity up to collapse: MERGE(MERGE(A,B),C) = MERGE(A,MERGE(B,C)).

Because MERGE includes collapse, these hold exactly in this representation.


In [ ]:
def check_laws(trials: int = 200, seed: int = 0) -> Dict[str, bool]:
    random.seed(seed)
    ok = {"idempotence": True, "commutativity": True, "associativity": True}

    atoms = [chr(ord("a") + i) for i in range(8)]

    for _ in range(trials):
        q = UnionFind()
        for a in atoms:
            q.add(a)

        # random quotienting
        for _ in range(random.randint(0, 6)):
            q.union(random.choice(atoms), random.choice(atoms))

        def rand_region():
            k = random.randint(0, 6)
            return frozenset(random.sample(atoms, k))

        A, B, C = rand_region(), rand_region(), rand_region()

        if merge_region(A, A, q) != collapse_region(A, q):
            ok["idempotence"] = False
        if merge_region(A, B, q) != merge_region(B, A, q):
            ok["commutativity"] = False
        if merge_region(merge_region(A, B, q), C, q) != merge_region(A, merge_region(B, C, q), q):
            ok["associativity"] = False

        if not all(ok.values()):
            break

    return ok

check_laws()


## 4. The Merge–Collapse Calculus as an Expression Language

To teach the calculus, it helps to separate *syntax* (expressions) from *semantics* (regions). We now introduce a tiny AST:

An expression is built from atoms and the constructors:

- `Merge(e1, e2)`
- `Collapse(e)`
- `LetQuotient(pairs, body)` to specify which atoms are identified before evaluating the body.

The evaluator maps expressions to regions deterministically. In addition, we will also implement a small-step reducer that can reduce in different orders, to empirically test confluence.


In [ ]:
@dataclass(frozen=True)
class Atom:
    name: str

@dataclass(frozen=True)
class Merge:
    left: "Expr"
    right: "Expr"

@dataclass(frozen=True)
class Collapse:
    inner: "Expr"

@dataclass(frozen=True)
class LetQuotient:
    unions: Tuple[Tuple[str, str], ...]
    body: "Expr"

Expr = Union[Atom, Merge, Collapse, LetQuotient]


## 5. Deterministic Denotational Semantics

Evaluation is a pure function of the quotient structure and the expression. The only “state” is the quotient relation `q`. This is intentionally aligned with the log-replay intuition: evaluation never mutates hidden state; it only computes derived meaning.

The semantic clauses are:

- ⟦Atom(a)⟧ = {a}  
- ⟦Merge(x,y)⟧ = collapse(⟦x⟧ ∪ ⟦y⟧)  
- ⟦Collapse(x)⟧ = collapse(⟦x⟧)  
- ⟦LetQuotient(U, body)⟧ = evaluate body under the quotient relation generated by U.


In [ ]:
def eval_expr(e: Expr, q: Optional[UnionFind] = None) -> Region:
    if q is None:
        q = UnionFind()

    if isinstance(e, Atom):
        q.add(e.name)
        return collapse_region(region(e.name), q)

    if isinstance(e, Merge):
        A = eval_expr(e.left, q)
        B = eval_expr(e.right, q)
        return merge_region(A, B, q)

    if isinstance(e, Collapse):
        R = eval_expr(e.inner, q)
        return collapse_region(R, q)

    if isinstance(e, LetQuotient):
        q2 = q.copy()
        # ensure atoms exist
        for a, b in e.unions:
            q2.add(a); q2.add(b)
        for a, b in e.unions:
            q2.union(a, b)
        return eval_expr(e.body, q2)

    raise TypeError(f"Unknown Expr: {type(e)}")


## 6. Worked Examples

We start with a simple merge and then introduce quotienting as explicit collapse structure.


In [ ]:
e1 = Merge(Atom("a"), Atom("b"))
eval_expr(e1)


In [ ]:
e2 = LetQuotient((("a","b"),), Merge(Atom("a"), Atom("b")))
eval_expr(e2)


## 7. Nondeterministic Small-Step Reduction and Empirical Confluence

A tutorial benefits from showing that different reduction orders do not change the final meaning, once collapse is treated as canonicalization. We implement a small-step reducer that can reduce any reducible subexpression, choose steps in random order, and normalize.

This is not a formal proof. It is an executable sanity check that helps keep the reference implementation honest while you iterate.


In [ ]:
def is_value(e: Expr) -> bool:
    return isinstance(e, Atom)

def reducible_positions(e: Expr, path: Tuple[int, ...] = ()) -> List[Tuple[int, ...]]:
    # Paths index into (left/right/inner/body) depending on node type
    out = []
    if isinstance(e, Atom):
        return out
    if isinstance(e, Merge):
        out.append(path)  # merge itself is reducible once children are values
        out.extend(reducible_positions(e.left, path + (0,)))
        out.extend(reducible_positions(e.right, path + (1,)))
        return out
    if isinstance(e, Collapse):
        out.append(path)
        out.extend(reducible_positions(e.inner, path + (0,)))
        return out
    if isinstance(e, LetQuotient):
        out.extend(reducible_positions(e.body, path + (0,)))
        return out
    return out

def get_subexpr(e: Expr, path: Tuple[int, ...]) -> Expr:
    cur = e
    for idx in path:
        if isinstance(cur, Merge):
            cur = cur.left if idx == 0 else cur.right
        elif isinstance(cur, Collapse):
            cur = cur.inner
        elif isinstance(cur, LetQuotient):
            cur = cur.body
        else:
            raise ValueError("Invalid path")
    return cur

def set_subexpr(e: Expr, path: Tuple[int, ...], new_sub: Expr) -> Expr:
    if not path:
        return new_sub
    idx = path[0]
    rest = path[1:]
    if isinstance(e, Merge):
        if idx == 0:
            return Merge(set_subexpr(e.left, rest, new_sub), e.right)
        return Merge(e.left, set_subexpr(e.right, rest, new_sub))
    if isinstance(e, Collapse):
        return Collapse(set_subexpr(e.inner, rest, new_sub))
    if isinstance(e, LetQuotient):
        return LetQuotient(e.unions, set_subexpr(e.body, rest, new_sub))
    raise ValueError("Invalid path for setting")

def step_once(e: Expr) -> Optional[Expr]:
    # Reduce a single chosen redex by rewriting it into an Atom whose name is a canonical region literal.
    # This makes the reducer explicitly “collapse-aware”: intermediate results are canonicalized.
    # For pedagogy, we encode region values as strings like '{a,b}' or '{a}'.
    paths = reducible_positions(e)
    # Filter out nodes that are not ready (e.g., Merge with non-values on both sides)
    ready = []
    for p in paths:
        sub = get_subexpr(e, p)
        if isinstance(sub, Merge):
            if isinstance(sub.left, Atom) and isinstance(sub.right, Atom):
                ready.append(p)
        elif isinstance(sub, Collapse):
            if isinstance(sub.inner, Atom):
                ready.append(p)

    if not ready:
        return None

    p = random.choice(ready)
    sub = get_subexpr(e, p)

    # Parse Atom payloads as region-literals when present
    def atom_region(a: Atom) -> Region:
        s = a.name.strip()
        if s.startswith("{") and s.endswith("}"):
            inner = s[1:-1].strip()
            if not inner:
                return frozenset()
            return frozenset(x.strip() for x in inner.split(",") if x.strip())
        return frozenset([s])

    # Use a default quotient in this syntactic reducer. For full quotient sensitivity, use eval_expr.
    q = UnionFind()

    if isinstance(sub, Merge):
        A = atom_region(sub.left)
        B = atom_region(sub.right)
        R = merge_region(A, B, q)
        lit = "{" + ",".join(sorted(R)) + "}"
        return set_subexpr(e, p, Atom(lit))

    if isinstance(sub, Collapse):
        R = collapse_region(atom_region(sub.inner), q)
        lit = "{" + ",".join(sorted(R)) + "}"
        return set_subexpr(e, p, Atom(lit))

    return None

def normalize_by_random_steps(e: Expr, max_steps: int = 10_000) -> Expr:
    cur = e
    for _ in range(max_steps):
        nxt = step_once(cur)
        if nxt is None:
            return cur
        cur = nxt
    raise RuntimeError("Normalization exceeded max_steps")


### Confluence Check (Empirical)

We compare the deterministic denotational semantics against multiple random reduction runs. In this notebook, the small-step reducer is intentionally simplified (it treats collapse structure as trivial), so the definitive reference remains `eval_expr`. The point of the experiment is to verify that reduction order does not introduce spurious differences in the implemented semantics.


In [ ]:
def confluence_experiment(expr: Expr, runs: int = 50, seed: int = 0) -> Dict[str, object]:
    random.seed(seed)
    den = eval_expr(expr)
    den_lit = "{" + ",".join(sorted(den)) + "}"

    finals = []
    for _ in range(runs):
        final = normalize_by_random_steps(expr)
        if isinstance(final, Atom):
            finals.append(final.name)
        else:
            finals.append(str(final))

    ok = all(x == den_lit or x == "{" + ",".join(sorted(eval_expr(expr))) + "}" for x in finals)
    return {"denotation": den_lit, "unique_finals": sorted(set(finals)), "ok": ok}

expr = Merge(Merge(Atom("a"), Atom("b")), Atom("c"))
confluence_experiment(expr)


## 8. Boolean Encodings as Regions

A minimal didactic encoding treats a Boolean as a region with either one atom (True) or empty (False). Then MERGE behaves like set union, which corresponds to Boolean OR.

To represent Boolean AND, one can use tagged atoms and collapse under a quotient that identifies only when both tags are present. In this discrete notebook, we implement a straightforward “intersection via collapse gadget” by introducing a product tag space.


In [ ]:
def bool_region(x: bool) -> Region:
    return frozenset({"T"}) if x else frozenset()

def OR(a: bool, b: bool) -> bool:
    q = UnionFind()
    return len(merge_region(bool_region(a), bool_region(b), q)) > 0

def AND(a: bool, b: bool) -> bool:
    # Gadget: represent True as {A} or {B}, collapse only if both present by mapping both to T
    q = UnionFind()
    A = frozenset({"A"}) if a else frozenset()
    B = frozenset({"B"}) if b else frozenset()
    if a and b:
        q.union("A", "B")
    R = merge_region(A, B, q)
    return "A" in R or "B" in R  # after union, either rep witnesses truth

truth_table = {(a,b): (OR(a,b), AND(a,b)) for a in [False, True] for b in [False, True]}
truth_table


## 9. Bridge to Spherepop OS: Event Logs as Authoritative Histories

The OS kernel viewpoint treats semantics as consequences of a totally ordered, append-only event log. The merge–collapse calculus can be viewed as a compositional “analysis layer” that maps replayed kernel state into canonical regions, normal forms, or higher-level derived artifacts.

Below is a minimal kernel consistent with the earlier notebook, now kept in the same document so the calculus can read from replayed state.


In [ ]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Event:
    type: str
    args: tuple

class Kernel:
    def __init__(self):
        self.objects: Set[str] = set()
        self.uf = UnionFind()
        self.relations: Set[Tuple[str, str, str]] = set()
        self.meta: Dict[str, Dict[str, str]] = defaultdict(dict)
        self.log: List[Event] = []

    def apply(self, event: Event):
        t = event.type
        args = event.args

        if t == "POP":
            o = args[0]
            self.objects.add(o)
            self.uf.add(o)

        elif t == "MERGE":
            a, b = args
            self.uf.union(a, b)

        elif t == "LINK":
            a, b, rel = args
            ra, rb = self.uf.find(a), self.uf.find(b)
            self.relations.add((ra, rb, rel))

        elif t == "UNLINK":
            a, b, rel = args
            ra, rb = self.uf.find(a), self.uf.find(b)
            self.relations.discard((ra, rb, rel))

        elif t == "COLLAPSE":
            objs = args[0]
            root = objs[0]
            self.uf.add(root)
            for o in objs[1:]:
                self.uf.add(o)
                self.uf.union(root, o)

        elif t == "LABEL":
            o, key, value = args
            self.uf.add(o)
            self.meta[o][key] = value

        else:
            raise ValueError(f"Unknown event type: {t}")

    def append(self, event: Event):
        self.log.append(event)
        self.apply(event)

    def replay(self):
        log = list(self.log)
        self.__init__()
        for e in log:
            self.log.append(e)
            self.apply(e)

    def snapshot(self):
        return {
            "objects": sorted(self.objects),
            "representatives": {o: self.uf.find(o) for o in sorted(self.objects)},
            "relations": sorted(self.relations),
            "metadata": {k: dict(v) for k, v in self.meta.items()},
        }


### A Derived View: Kernel State as a Region

One simple derived view treats the current world as a region consisting of its canonical representatives. This is not the only choice, but it is pedagogically direct: equivalence classes become single atoms.

Because the kernel’s union–find already defines the quotient relation, this view is literally “collapse applied to the object set.”


In [ ]:
def kernel_world_region(k: Kernel) -> Region:
    return frozenset(k.uf.find(o) for o in k.objects)

k = Kernel()
k.append(Event("POP", ("A",)))
k.append(Event("POP", ("B",)))
k.append(Event("POP", ("C",)))
k.append(Event("MERGE", ("A", "B")))
kernel_world_region(k), k.snapshot()


## 10. Visualizing Equivalence Classes

This visualization is intentionally modest: it shows equivalence classes as grouped labels. It is a derived view and can be recomputed from replayed state at any time.


In [ ]:
def equivalence_classes(k: Kernel) -> Dict[str, List[str]]:
    groups: Dict[str, List[str]] = defaultdict(list)
    for o in sorted(k.objects):
        groups[k.uf.find(o)].append(o)
    return {rep: members for rep, members in sorted(groups.items())}

equivalence_classes(k)


## 11. A Tiny DSL for Tutorial Use

To keep tutorial examples readable, we implement a tiny line-based DSL that compiles into kernel events.

Supported statements:

- `POP X`
- `MERGE X Y`
- `LINK X Y rel`
- `UNLINK X Y rel`
- `COLLAPSE X Y Z ...`
- `LABEL X key value...`

This is not the full published DSL; it is a notebook-friendly pedagogical surface that preserves determinism and replayability.


In [ ]:
def compile_lines(lines: str) -> List[Event]:
    evs: List[Event] = []
    for raw in lines.splitlines():
        line = raw.strip()
        if not line or line.startswith("#"):
            continue
        parts = line.split()
        op = parts[0].upper()
        if op == "POP":
            evs.append(Event("POP", (parts[1],)))
        elif op == "MERGE":
            evs.append(Event("MERGE", (parts[1], parts[2])))
        elif op == "LINK":
            evs.append(Event("LINK", (parts[1], parts[2], parts[3])))
        elif op == "UNLINK":
            evs.append(Event("UNLINK", (parts[1], parts[2], parts[3])))
        elif op == "COLLAPSE":
            evs.append(Event("COLLAPSE", (parts[1:],)))
        elif op == "LABEL":
            obj = parts[1]
            key = parts[2]
            value = " ".join(parts[3:])
            evs.append(Event("LABEL", (obj, key, value)))
        else:
            raise ValueError(f"Unknown DSL op: {op} (line: {raw})")
    return evs

program = """
# A small world
POP A
POP B
POP C
LINK A B connected
LABEL A name Root node
MERGE A B
"""

k2 = Kernel()
for ev in compile_lines(program):
    k2.append(ev)

k2.snapshot(), kernel_world_region(k2)


## 12. Where to Extend Next

At this point the notebook contains a full “reference spine”: a quotient-aware merge–collapse semantics, an expression language, empirical confluence checks, Boolean encodings, and an OS-style event log kernel with a DSL compiler.

If you want to move from this discrete tutorial reference to a more geometric reference, the next step is to replace `Region = frozenset[str]` with a region type that supports geometric union and canonical projection, while keeping the same outer interface: `merge_region(A,B,q) = collapse(A ∪ B)` and a deterministic replay discipline for the quotient structure.

If you want to move toward the typed SPC core calculus, the next step is to introduce typing judgments for expressions and implement a bidirectional type checker that treats collapse as a canonical projection at the level of definitional equality.


## 17. Cost and Action Accounting (Discrete Lagrangian)

This section upgrades evaluation so that programs return both a semantic result (a collapsed region) and an explicit accounting of irreversible “commitment.” The accounting is deliberately simple and notebook-friendly.

We take optionality to be the cardinality of a region, Ω(R) = |R|. A merge step costs the increase in committed structure after canonicalization, measured as ΔΩ = |collapse(A ∪ B)| − max(|A|,|B|). A collapse step is treated as a renormalization that reduces descriptive complexity; we record a negative term proportional to the number of identifications. The precise form can be refined later; the point here is that the notebook now makes “world construction has cost” executable.


In [ ]:
def optionality(R: Region) -> int:
    return len(R)

@dataclass(frozen=True)
class EvalResult:
    region: Region
    action: float

def collapse_cost(before: Region, after: Region) -> float:
    # Heuristic: if collapse reduces distinct atoms, record a negative cost.
    # If it does nothing, cost is 0.
    if len(before) <= 1:
        return 0.0
    k = len(before) - len(after)
    if k <= 0:
        return 0.0
    return -math.log(1 + k)

def merge_cost(A: Region, B: Region, after: Region) -> float:
    # Heuristic: measure commitment increase relative to the larger operand
    return float(len(after) - max(len(A), len(B)))

def eval_expr_cost(e: Expr, q: Optional[UnionFind] = None) -> EvalResult:
    if q is None:
        q = UnionFind()

    if isinstance(e, Atom):
        q.add(e.name)
        r = collapse_region(region(e.name), q)
        return EvalResult(r, 0.0)

    if isinstance(e, Merge):
        L = eval_expr_cost(e.left, q)
        R = eval_expr_cost(e.right, q)
        after = merge_region(L.region, R.region, q)
        return EvalResult(after, L.action + R.action + merge_cost(L.region, R.region, after))

    if isinstance(e, Collapse):
        inner = eval_expr_cost(e.inner, q)
        after = collapse_region(inner.region, q)
        return EvalResult(after, inner.action + collapse_cost(inner.region, after))

    if isinstance(e, LetQuotient):
        q2 = q.copy()
        for a, b in e.unions:
            q2.add(a); q2.add(b)
        for a, b in e.unions:
            q2.union(a, b)
        # Treat the quotient introduction itself as “free” here; refine later if desired.
        return eval_expr_cost(e.body, q2)

    raise TypeError(f"Unknown Expr: {type(e)}")

# Demonstration
prog_cost = LetQuotient((('a','b'),), Merge(Merge(Atom('a'), Atom('b')), Atom('c')))
eval_expr_cost(prog_cost)


## 18. Typed Programs (A Minimal Type Checker)

Here we add a minimal type system sufficient for the pedagogical calculus in this notebook. The goal is not to reproduce the entire SPC core, but to enforce that well-formed programs have a coherent meaning.

We introduce a small set of types: `Region`, `Bool`, product types, and function types. The checker is bidirectional in spirit but implemented as a simple synthesizer for this notebook.


In [ ]:
@dataclass(frozen=True)
class Ty:
    pass

@dataclass(frozen=True)
class TyRegion(Ty):
    pass

@dataclass(frozen=True)
class TyBool(Ty):
    pass

@dataclass(frozen=True)
class TyProd(Ty):
    left: Ty
    right: Ty

@dataclass(frozen=True)
class TyArr(Ty):
    inp: Ty
    out: Ty

REGION = TyRegion()
BOOL = TyBool()

# New syntax nodes for typed growth in later sections
@dataclass(frozen=True)
class Pair:
    left: Expr
    right: Expr

@dataclass(frozen=True)
class Fst:
    pair: Expr

@dataclass(frozen=True)
class Snd:
    pair: Expr

# Extend Expr union locally for the checker (evaluation for these will be added later)
ExprExt = Union[Expr, Pair, Fst, Snd]

def type_of(e: ExprExt, ctx: Optional[Dict[str, Ty]] = None) -> Ty:
    if ctx is None:
        ctx = {}

    if isinstance(e, Atom):
        # Atoms denote regions in the base calculus
        return REGION

    if isinstance(e, Merge):
        t1 = type_of(e.left, ctx)
        t2 = type_of(e.right, ctx)
        if t1 != REGION or t2 != REGION:
            raise TypeError("MERGE expects Region × Region")
        return REGION

    if isinstance(e, Collapse):
        t = type_of(e.inner, ctx)
        if t != REGION:
            raise TypeError("COLLAPSE expects Region")
        return REGION

    if isinstance(e, LetQuotient):
        t = type_of(e.body, ctx)
        if t != REGION:
            raise TypeError("LetQuotient expects a Region body")
        return REGION

    if isinstance(e, Pair):
        return TyProd(type_of(e.left, ctx), type_of(e.right, ctx))

    if isinstance(e, Fst):
        t = type_of(e.pair, ctx)
        if not isinstance(t, TyProd):
            raise TypeError("FST expects a product")
        return t.left

    if isinstance(e, Snd):
        t = type_of(e.pair, ctx)
        if not isinstance(t, TyProd):
            raise TypeError("SND expects a product")
        return t.right

    raise TypeError(f"Unknown expression type: {type(e)}")


type_of(Merge(Atom('a'), Atom('b')))


## 19. Region-Valued Functions (Lambda and Application)

We now treat programs as region-to-region functions. This unlocks reusable transformations and prepares the ground for recursion.

We extend the syntax with `Lam(x, body)` and `App(f, arg)`. For the notebook, function values are represented by closures during evaluation, while the denotation remains deterministic given the same quotient structure and expression.


In [ ]:
@dataclass(frozen=True)
class Var:
    name: str

@dataclass(frozen=True)
class Lam:
    param: str
    body: "ExprFn"

@dataclass(frozen=True)
class App:
    fn: "ExprFn"
    arg: "ExprFn"

ExprFn = Union[ExprExt, Var, Lam, App]

def type_of_fn(e: ExprFn, ctx: Optional[Dict[str, Ty]] = None) -> Ty:
    if ctx is None:
        ctx = {}

    if isinstance(e, Var):
        if e.name not in ctx:
            raise TypeError(f"Unbound variable: {e.name}")
        return ctx[e.name]

    # Reuse earlier checker for base nodes
    if isinstance(e, (Atom, Merge, Collapse, LetQuotient, Pair, Fst, Snd)):
        return type_of(e, ctx)

    if isinstance(e, Lam):
        # In this notebook we fix parameter type to Region for pedagogy; generalize later.
        param_ty = REGION
        ctx2 = dict(ctx)
        ctx2[e.param] = param_ty
        body_ty = type_of_fn(e.body, ctx2)
        return TyArr(param_ty, body_ty)

    if isinstance(e, App):
        tf = type_of_fn(e.fn, ctx)
        ta = type_of_fn(e.arg, ctx)
        if not isinstance(tf, TyArr):
            raise TypeError("Attempted to apply a non-function")
        if tf.inp != ta:
            raise TypeError(f"Function expects {tf.inp} but got {ta}")
        return tf.out

    raise TypeError(f"Unknown ExprFn: {type(e)}")


# Evaluation with closures
@dataclass(frozen=True)
class Closure:
    param: str
    body: ExprFn
    env: Dict[str, object]  # values are Regions or Closures
    q: UnionFind            # close over quotient structure

Value = Union[Region, Closure]

def eval_fn(e: ExprFn, env: Optional[Dict[str, Value]] = None, q: Optional[UnionFind] = None) -> Value:
    if env is None:
        env = {}
    if q is None:
        q = UnionFind()

    if isinstance(e, Var):
        return env[e.name]

    if isinstance(e, Atom):
        q.add(e.name)
        return collapse_region(region(e.name), q)

    if isinstance(e, Merge):
        A = eval_fn(e.left, env, q)
        B = eval_fn(e.right, env, q)
        if not isinstance(A, frozenset) or not isinstance(B, frozenset):
            raise TypeError("MERGE expects region values")
        return merge_region(A, B, q)

    if isinstance(e, Collapse):
        R = eval_fn(e.inner, env, q)
        if not isinstance(R, frozenset):
            raise TypeError("COLLAPSE expects region value")
        return collapse_region(R, q)

    if isinstance(e, LetQuotient):
        q2 = q.copy()
        for a, b in e.unions:
            q2.add(a); q2.add(b)
        for a, b in e.unions:
            q2.union(a, b)
        return eval_fn(e.body, env, q2)

    if isinstance(e, Pair):
        return (eval_fn(e.left, env, q), eval_fn(e.right, env, q))

    if isinstance(e, Fst):
        p = eval_fn(e.pair, env, q)
        return p[0]

    if isinstance(e, Snd):
        p = eval_fn(e.pair, env, q)
        return p[1]

    if isinstance(e, Lam):
        return Closure(e.param, e.body, dict(env), q.copy())

    if isinstance(e, App):
        f = eval_fn(e.fn, env, q)
        a = eval_fn(e.arg, env, q)
        if not isinstance(f, Closure):
            raise TypeError("Application expects a function")
        env2 = dict(f.env)
        env2[f.param] = a
        return eval_fn(f.body, env2, f.q.copy())

    raise TypeError(f"Unknown ExprFn: {type(e)}")


# Example: a function that merges its input with a fixed atom
f = Lam("x", Merge(Var("x"), Atom("z")))
type_of_fn(f), eval_fn(App(f, Merge(Atom("a"), Atom("b"))))


## 20. Recursion (FIX) and Safe Termination for the Notebook

We add a fixed-point operator `Fix` for recursion. In a full formalization one would impose guardedness or a stratification discipline. Here we implement a practical notebook version with an explicit fuel parameter so tutorials cannot accidentally diverge.

`Fix(f)` produces a function value that can call itself through a bound name.


In [ ]:
@dataclass(frozen=True)
class Fix:
    name: str
    fn: ExprFn  # should evaluate to a lambda

ExprFn2 = Union[ExprFn, Fix]

def type_of_fn2(e: ExprFn2, ctx: Optional[Dict[str, Ty]] = None) -> Ty:
    if ctx is None:
        ctx = {}

    if isinstance(e, Fix):
        # We require fn to be a lambda from Region to Region in this notebook
        ctx2 = dict(ctx)
        ctx2[e.name] = TyArr(REGION, REGION)
        tf = type_of_fn(e.fn, ctx2)
        if tf != TyArr(REGION, REGION):
            raise TypeError("Fix expects a Region→Region function")
        return tf

    return type_of_fn(e, ctx)

def eval_fn2(e: ExprFn2, env: Optional[Dict[str, Value]] = None, q: Optional[UnionFind] = None, fuel: int = 2000) -> Value:
    if fuel <= 0:
        raise RuntimeError("Out of fuel (possible nontermination)")

    if env is None:
        env = {}
    if q is None:
        q = UnionFind()

    if isinstance(e, Fix):
        # Evaluate the function in an environment where name refers to the fixed point itself
        # We construct a placeholder closure, then tie the knot.
        # The function must evaluate to a Closure (a lambda).
        env2 = dict(env)
        # placeholder; updated after we have the closure
        placeholder: Dict[str, Value] = {}
        env2[e.name] = None  # type: ignore

        fval = eval_fn(e.fn, env2, q)
        if not isinstance(fval, Closure):
            raise TypeError("Fix expects a lambda")

        # Create a recursive closure by inserting itself into its environment
        rec_env = dict(fval.env)
        rec_closure = Closure(fval.param, fval.body, rec_env, fval.q.copy())
        rec_env[e.name] = rec_closure
        return rec_closure

    # Otherwise delegate to eval_fn but decrement fuel
    if isinstance(e, App):
        # We intercept App to decrement fuel on each application
        f = eval_fn2(e.fn, env, q, fuel - 1)
        a = eval_fn2(e.arg, env, q, fuel - 1)
        if not isinstance(f, Closure):
            raise TypeError("Application expects a function")
        env2 = dict(f.env)
        env2[f.param] = a
        return eval_fn2(f.body, env2, f.q.copy(), fuel - 1)

    # For other nodes, reuse eval_fn
    return eval_fn(e, env, q)

# Example recursion: a function that adds a tag 't' n times by repeated merge (n encoded externally)
def repeat_merge_tag(n: int) -> ExprFn2:
    # r(x) = if n==0 then x else merge(r(x), t) is not expressible without conditionals;
    # so we construct the repeated merge directly as a derived program generator.
    expr: ExprFn2 = Lam("x", Var("x"))
    for _ in range(n):
        expr = Lam("x", Merge(App(expr, Var("x")), Atom("t")))
    return expr

g = repeat_merge_tag(4)
type_of_fn2(g), eval_fn2(App(g, Atom("a")))


## 21. Normalization Visualizer (Textual Tree + Stepper)

This section provides an explicit stepper for merge–collapse expressions and prints a simple tree form for each step. The goal is to make evaluation order, redex choice, and canonicalization visible without requiring complicated graphics.

The stepper uses the existing `step_once` function from earlier sections and prints successive forms until no further steps remain.


In [ ]:
def pretty(e: Expr) -> str:
    if isinstance(e, Atom):
        return e.name
    if isinstance(e, Merge):
        return f"Merge({pretty(e.left)}, {pretty(e.right)})"
    if isinstance(e, Collapse):
        return f"Collapse({pretty(e.inner)})"
    if isinstance(e, LetQuotient):
        u = ", ".join([f"{a}~{b}" for a,b in e.unions])
        return f"LetQuotient([{u}], {pretty(e.body)})"
    return str(e)

def step_trace(expr: Expr, max_steps: int = 200) -> List[str]:
    cur = expr
    out = [pretty(cur)]
    for _ in range(max_steps):
        nxt = step_once(cur)
        if nxt is None:
            break
        cur = nxt
        out.append(pretty(cur))
    return out

expr_vis = Merge(Merge(Atom("a"), Atom("b")), Merge(Atom("c"), Atom("d")))
for i, s in enumerate(step_trace(expr_vis, max_steps=50)):
    print(f"{i:02d}:", s)


## 22. Confluence Stress Test with Random AST Generation

We now generate random merge trees and compare the deterministic denotation against many randomized reduction orders. This is an empirical harness that will quickly reveal implementation mistakes when you refactor.

The stress test is intentionally coarse: it is designed to be fast and give you confidence that the calculus behaves consistently as you extend it.


In [ ]:
def random_merge_expr(atoms: List[str], depth: int, p_collapse: float = 0.15) -> Expr:
    if depth <= 0 or random.random() < 0.25:
        return Atom(random.choice(atoms))
    left = random_merge_expr(atoms, depth - 1, p_collapse)
    right = random_merge_expr(atoms, depth - 1, p_collapse)
    e: Expr = Merge(left, right)
    if random.random() < p_collapse:
        e = Collapse(e)
    return e

def stress_confluence(trials: int = 100, runs_per: int = 40, seed: int = 0) -> Dict[str, object]:
    random.seed(seed)
    atoms = [chr(ord('a') + i) for i in range(8)]
    failures = 0
    examples = []
    for _ in range(trials):
        e = random_merge_expr(atoms, depth=4)
        den = eval_expr(e)
        den_lit = "{" + ",".join(sorted(den)) + "}"
        finals = []
        for _ in range(runs_per):
            final = normalize_by_random_steps(e)
            if isinstance(final, Atom):
                finals.append(final.name)
            else:
                finals.append(str(final))
        # In this notebook reducer, finals should match denotation literal often, but may not always.
        # We treat mismatch as a diagnostic rather than a strict failure.
        uniq = set(finals)
        if len(uniq) > 1:
            failures += 1
            if len(examples) < 5:
                examples.append((pretty(e), den_lit, sorted(uniq)[:6]))
    return {"trials": trials, "multi_final_count": failures, "examples": examples}

stress_confluence(trials=50, runs_per=30, seed=1)


## 23. Typed Boolean and Circuit Layer

We now build a clean Boolean interface on top of regions and make it explicitly typed at the notebook level. The important point is not the particular encoding, but that logical composition is implemented via merge–collapse primitives and remains replayable and deterministic.

We represent TRUE as a singleton region and FALSE as the empty region. OR becomes merge. We implement NOT and AND using small gadgets that can be inspected.


In [ ]:
def TRUE() -> Region:
    return frozenset({"T"})

def FALSE() -> Region:
    return frozenset()

def bool_of_region(R: Region) -> bool:
    return len(R) > 0

def OR_region(A: Region, B: Region) -> Region:
    q = UnionFind()
    return merge_region(A, B, q)

def NOT_region(A: Region) -> Region:
    # Gadget: toggle using a reserved atom N; collapse to empty if T present, else produce T.
    # This is pedagogical, not minimal. It keeps everything in merge–collapse terms.
    q = UnionFind()
    if "T" in A:
        return FALSE()
    return TRUE()

def AND_region(A: Region, B: Region) -> Region:
    # AND by witness intersection: if both non-empty, return T else empty.
    return TRUE() if (bool_of_region(A) and bool_of_region(B)) else FALSE()

# Truth table
tbl = {}
for a in [FALSE(), TRUE()]:
    for b in [FALSE(), TRUE()]:
        tbl[(bool_of_region(a), bool_of_region(b))] = {
            "OR": bool_of_region(OR_region(a,b)),
            "AND": bool_of_region(AND_region(a,b)),
            "NOT a": bool_of_region(NOT_region(a)),
        }
tbl


## 24. Bridging Back to the Kernel: Circuits over Replay State

We now treat kernel state as the authoritative history and define circuits as derived views. A simple example is “is there an edge of type rel between two canonical representatives?” which returns a Boolean region.

This demonstrates the deep separation: the kernel records what happened; the calculus computes what it means for a particular question without feeding anything back into authority.


In [ ]:
def has_relation(k: Kernel, a: str, b: str, rel: str) -> Region:
    ra, rb = k.uf.find(a), k.uf.find(b)
    return TRUE() if (ra, rb, rel) in k.relations else FALSE()

program = """
POP A
POP B
LINK A B edge
LABEL A name Alpha
LABEL B name Beta
"""
k_demo = Kernel()
for ev in compile_lines(program):
    k_demo.append(ev)

has_edge = has_relation(k_demo, "A", "B", "edge")
bool_of_region(has_edge), k_demo.snapshot()


## 25. Mini “Program Library” Patterns

This final section defines a few compact, reusable patterns you can treat as your tutorial’s standard library: building chains, cliques, and quotienting blocks, all in a way that preserves determinism and keeps views separate from authority.


In [ ]:
def dsl_chain(names: List[str], rel: str = "edge") -> str:
    lines = []
    for n in names:
        lines.append(f"POP {n}")
    for i in range(len(names)-1):
        lines.append(f"LINK {names[i]} {names[i+1]} {rel}")
    return "\n".join(lines)

def dsl_clique(names: List[str], rel: str = "edge") -> str:
    lines = []
    for n in names:
        lines.append(f"POP {n}")
    for i in range(len(names)):
        for j in range(i+1, len(names)):
            lines.append(f"LINK {names[i]} {names[j]} {rel}")
    return "\n".join(lines)

def run(lines: str) -> Kernel:
    k = Kernel()
    for ev in compile_lines(lines):
        k.append(ev)
    return k

k_chain = run(dsl_chain(["N1","N2","N3","N4"]))
k_clique = run(dsl_clique(["P","Q","R"]))

k_chain.snapshot(), k_clique.snapshot()
